In [9]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import pickle
import pathlib
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback

openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
pdf_reader = PdfReader("./documentations/examples/companies/garmin/Enduro_2_OM_EN-US.pdf")

In [4]:
text = ""

for page in pdf_reader.pages:
    text += page.extract_text()
    
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
chunks = text_splitter.split_text(text=text)

In [5]:
chunks

['ENDURO™ 2\nOwner’s Manual© 2022 Garmin Ltd. or its subsidiaries\nAll rights reserved. Under the copyright laws, this manual may not be copied, in whole or in part, without the written consent of Garmin. Garmin reserves the right to change \nor improve its products and to make changes in the content of this manual without obligation to notify any person or organization of such changes or improvements. Go to \nwww.garmin.com  for current updates and supplemental information concerning the use of this product.\nGarmin®, the Garmin logo, Enduro™, ANT®, ANT+®, Approach®, Auto Lap®, Auto Pause®, Edge®, inReach®, QuickFit®, TracBack®, VIRB®, Virtual Partner®, and Xero® are trademarks of \nGarmin Ltd. or its subsidiaries, registered in the USA and other countries. Body Battery™, Connect IQ™, Firstbeat Analytics™, Garmin AutoShot™, Garmin Connect™, Garmin Explore™,',
 'Garmin Ltd. or its subsidiaries, registered in the USA and other countries. Body Battery™, Connect IQ™, Firstbeat Analytics™,

In [6]:
# Embeddings
pdf_name = "mypdf.pdf"
store_name = pathlib.Path(pdf_name).stem

if os.path.exists(f"{store_name}.pickle"):
    with open(f"{store_name}.pickle", "rb") as f:
        VectorStore = pickle.load(f)
else:
    embeddings = OpenAIEmbeddings()
    VectorStore = FAISS.from_texts(chunks, embedding=embeddings)    
    with open(f"{store_name}.pickle", "wb") as f:
        pickle.dump(VectorStore, f)

### Accept user questions/query

In [13]:
query = input("Enter your query :")
if query:
    docs = VectorStore.similarity_search(query=query, k=3)
    
    llm = OpenAI(model_name='gpt-3.5-turbo')
    chain = load_qa_chain(llm=llm, chain_type="stuff")
    with get_openai_callback() as cb:
        response = chain.run(input_documents=docs, question=query)
        print(cb)
    print(response)



/home/leo/miniconda3/envs/chat_docs_env/lib/python3.11/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/leo/miniconda3/envs/chat_docs_env/lib/python3.11/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


Tokens Used: 795
	Prompt Tokens: 648
	Completion Tokens: 147
Successful Requests: 1
Total Cost (USD): $0.0012660000000000002
To reset your Garmin watch, follow these steps:
1. From the watch face, hold MENU.
2. Select System > Reset.
3. Choose the option that best suits your needs:
   - To reset all settings to the factory default values and save user-entered information and activity history, select Reset Default Settings.
   - To delete all activities from the history, select Delete All Activities.
   - To reset distance and time totals, select Reset Totals.
   - To reset all settings to the factory default values and delete user-entered information and activity history, select Delete Data and Reset Settings.
   
Please note that if you have set up Garmin Pay or have music stored on your watch, these options will also be affected.
